# Notebook 3 - working with processes / threads 
-----------------------------------------------------------------------

<br>


## Table of Content <a id="toc"></a>

1. **[Multiprocessing (and refactoring)](#8)**  
   <br>
   
2. **[Numba and parallelization](#9)**  
    2.1. [automatic parallelization](#2.1)  
    2.2. [explicit parallelization (prange)](#2.2)  
    2.3. [controlling the number of threads used](#2.3)  
    <br>
    
**Supplementary material:**  
   * Annex 1 - [Parallelization of pairwise distance computation with multiprocess](#annexa)  
   * Annex 2 - [Parallelization of pairwise distance computation with numba](#annexb)  


<br>

## 1 Multiprocessing (and refactoring) <a id='8'></a>

We can take advantage of the multiple cores available on our computers by using the **`multiprocessing`** module. 

In this approach, separate __processes__ are used, __not threads__. 

The use of threads is generally blocked by Python because of the "*Global Interpreter Lock*". This was a necessary design feature as a trade-off for the enormous flexibility in memory management that Python makes possible. This means that there is no shared memory when using multiprocessing, and thus the individual tasks must be independent.

`multiprocessing` generally works well with lists, where one maps a function to each element of the list and these operations are computed as separated processes, on separate cores per element of the list. 

Indeed, any kind of parallelization technique is really only worth it if the task you want to do is actually *parallelizable*. It is sometimes hard to judge what is and is not easily parallelizable, and can often require that you refactor your code quite a bit.

A rule of thumb for whether parallelization is possible is to evaluate whether the task can be divided into subtasks which: 
1. **Do not depend on each others results.**
2. Are very similar.
3. Use independent parts of the data.

Point 1. is the most important, the others are helpful but not entirely necessary.

<br>

**Example:** consider our function to compute an integral from the previous lesson:

* Here is the code in **pure python**.

In [ ]:
def f_native(x):
    return x ** 2 - x

def integrate_f_native(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_native(a + i * dx)
    return s * dx

print(integrate_f_native(0, 2, 100))
%timeit -n 3 -r 7 _ = integrate_f_native(0, 2, 1000000)

Ideally, we would like to reduce this something that looks like:

```python
for i in range(len(data)):
    result[i] = function(data[i])

```

Equivalent to:
```python
map(function, data)
```

So, we apply a `function` to each element (`data[i]`) of `data`.  
The game is thus to re-write it slightly so it fits this template.

In [ ]:
# Let's work outside the function and focus on the main loop:

a = 0
b = 2
N = 1000000
dx = (b - a) / N

data = list(range(N))

def f2(i):
    x = a + i*dx
    return x ** 2 - x

result = map(f2, data)

# Equivalent to
# for i in range(N):
#     result[i] = f_native(data[i])

final_result = sum(result) * dx
print(final_result)

Now, everything is ready for us to use `multiprocessing`.  
The simplest usage is to open up a pool of processes using the `with` keyword:

In [ ]:
import multiprocessing as mp

with mp.Pool(2) as pool:
    result = pool.map(f2, data)
    
final_result2 = sum(result) * dx
print(final_result2)

Ok so we get the same result when splitting the task on 2 processes, but does it perform faster?

In [ ]:
%timeit -n 3 -r 7 list(map(f2, data))

with mp.Pool(2) as pool:
    %timeit -n 3 -r 7 pool.map(f2, data)

Mhmm, what? the multiprocessing version is actually slower...

This is due to the fact that opening, closing, and communicating data to and from processes are costly operation.
In other words, **multiprocessing a task has some overhead**, and it therefore tends to work better with a few long tasks than with a lot of very small ones (note: each parallelization techniques have different overhead and react differently to this).

For instance, let's try with a few long tasks:
* Our "long" task will be to run the `integrate_f_native()` function between 0 and 1, with 400 thousand points.  
  This takes around 0.06 seconds.

In [ ]:
%timeit -n 3 -r 3 integrate_f_native(0, 1, 4 * 10**5)

In [ ]:
# Around 0.06 seconds per task
def task(i):
    return integrate_f_native(0, i, 4 * 10**5)

# 100 tasks to perform.
data = list(range(1, 101))

# Serial execution: ~6 seconds.
%time _ = list(map(task, data))

In [ ]:
with mp.Pool(2) as pool:
    %time  pool.map(task, data)

With such long tasks, the overhead is lower than the gained time.  
Indeed, on the basis of 0.06 seconds per task, we would expect 100 tasks on 2 processes to take ~ 3 seconds, so we have ~0.45 seconds of overhead here.

<br>

Let's now **vary the number of processes** to see whether adding more processors (CPUs) allows to further speed-up the computing.

In [ ]:
for number_of_processors in [1, 2, 4, 8]:
    print(number_of_processors)
    with mp.Pool(number_of_processors) as pool:
        %time pool.map(task, data)

> Note: as we increase the number of processes, the overhead increases and after some value this
> actually hurts the overall performance.
>
> * **Multiprocessing works better when the individual tasks are longer.**


<br>

<div class="alert alert-block alert-success">

## Exercise 3.1

See the dedicated `exercises_course2.ipynb` notebook.

<div>

<br>
<br>

[Back to ToC](#toc)

## 2. Numba and parallelization  <a id='9'></a>

It is possible to provide a `numba` function to `mp.pool`, but `numba` already provides what's necessary to parallelize code.
* By setting **`parallel=True`** when calling `@jit` (in no-python mode), numba will attempt
  to automatically parallelize your code.
* In particular, by default, it works on the array operations.

<br>

### 2.1 automatic parallelization <a id='2.1'></a>


In [ ]:
import numpy as np
from numba import njit

def integrate_f(a, b, N):
    dx = (b - a) / N
    X = np.arange(a,b,dx)
    return ( X**2 - X ).sum() * dx

# njit is a shortcut for jit(nopython=True).
# The code does not change, so no need to re-write it; just give the function to njit.
integrate_f_numba = njit(integrate_f)
integrate_f_numba_parallel = njit(integrate_f , parallel=True)

# Check that we get similar results: (additionally, this lets numba do the compilation now).
print( "native         :", integrate_f(0,2,100) )
print( "numba          :", integrate_f_numba(0,2,100) )
print( "numba parallel :", integrate_f_numba_parallel(0,2,100) )

In [ ]:
# Now let's time it.
N = 10**7
print( "native         :")
%timeit integrate_f(0,2,N) 
print( "numba          :")
%timeit integrate_f_numba(0,2,N)
print( "numba parallel :")
%timeit integrate_f_numba_parallel(0,2,N)

So the basic numba seems less efficient than numpy, but the parallel version is showing quite a speedup!

Here, numba was able to parallelize the `np.arange`, all the array operations, and the `sum()`, so actually almost all the code.


<br>

[Back to ToC](#toc)

### 2.2 explicit parallelization (prange) <a id='2.2'></a>

In [ ]:
def integrate_f2(a, b, N):
    dx = (b - a) / N
    s = 0
    for i in range(N):
        x = a + i*dx
        s += x**2 - x
    return s * dx

integrate_f2_numba = njit(integrate_f2)
integrate_f2_numba_parallel = njit(integrate_f2,parallel=True)


# Check that we get similar results: (+, this let's numba do the compilation now)
print("native         :", integrate_f2(0,2,100))
print("numba          :", integrate_f2_numba(0,2,100))
print("numba parallel :", integrate_f2_numba_parallel(0,2,100))

In [ ]:
from numba import prange 

@njit(parallel=True)
def integrate_f2_numba_parallel(a, b, N):
    dx = (b - a) / N
    s = 0
    for i in prange(N):
        x = a + i*dx
        s += x**2 - x
    return s * dx

integrate_f2_numba_parallel(0, 2, 100) 

In [ ]:
# Now let's time it
N = 10**7
print("numba from native         :")
%timeit -n 10 -r 7 integrate_f2_numba(0, 2, N)

print("numba parallel with prange:")
%timeit -n 10 -r 7 integrate_f2_numba_parallel(0, 2, N)

Let's use a bit more data to compare the 2 parallel versions (auto and manual):

In [ ]:
N = 10**8
print("numba parallel auto:")
%timeit -n 3 -r 7 integrate_f_numba_parallel(0, 2, N)

print("numba parallel with prange:")
%timeit -n 3 -r 7 integrate_f2_numba_parallel(0, 2, N)

The two results are similar. The one you end up using will depend on the structure of your problem and the shape of your code.

<br>

[Back to ToC](#toc)

### 2.3 controlling the number of threads used <a id="2.3"></a>

Up until now, we have let numba use its default number of threads.

In [ ]:
import numba
numba.config.NUMBA_DEFAULT_NUM_THREADS

To control the number of threads, just use the `set_num_threads` function:

In [ ]:
from numba import set_num_threads

N = 10**8

# Max number of threads.
print("default number of threads")
%timeit -n 3 -r 7 integrate_f2_numba_parallel(0, 2, N)

for num_thread in range(2, 8):
    print(num_thread)
    set_num_threads(num_thread)
    %timeit -n 3 -r 7 integrate_f2_numba_parallel(0, 2, N)

To go further, we recommend you have a look at [numba documentation on parallelization](https://numba.pydata.org/numba-doc/latest/user/parallel.html) which explains what can, and what cannot be parallelized, and how to diagnose the automatic parallelization process.


<br>
<br>
<br>

[Back to ToC](#toc)

# Additional material
------------------------------

## Annex 1 - parallelization of pairwise distance computation with multiprocess <a id='annexa'></a>

In [ ]:
import numpy as np 

def pairwise_distance_numpy(X):

    num_vectors = X.shape[0]
    num_measurements = X.shape[1] 
    D = np.empty((num_vectors, num_vectors), dtype=np.float64)
    
    for i in range(num_vectors):
        for j in range(num_vectors):
            d = np.square(np.subtract(X[i], X[j]))
            D[i, j] = np.sqrt(np.sum(d))
            
    return(D)

Right now, this function operates onto a whole array. but ideally, we would like to reduce this something that looks like:

```python
for i in range(len(data)):
    result[i] = function(data[i])
```

Equivalent to:

```python
map(function,data)
```

So, we apply a `function` to each element (`data[i]`) of `data`.

**Question:** how can we go from the `pairwise_distance_numpy` function to this? what would be `function`? `data`? 

<br>

<br>

<br>

<br>

<br>

 ... don't scroll - spoilers ahead ...

<br>

<br>

<br>

<br>

<br>

<br>

<br>

So, my proposition to solve this (not the only one possible, maybe not even the best) is that :
 1. the `function` is computing distance between 2 vectors
 2. the `data[i]` is a couple of vector
 3. consequently, `data` is a list of couples of vectors.


I will even go one (small) step further, and rather than keeping the whole vectors in data, I will just keep the vector indexes

In [ ]:
# Generate 200 vectors with 100 measurements each 
data = np.random.uniform(size=(200,100))

In [ ]:
def pairwise_list_I(X):
    """ create a list of the pairs of vector index we have to compute distances for (ie. all possible pair of indexes)"""
    list_of_tuples = list()
    
    num_vectors = X.shape[0]
    num_measurements = X.shape[1] 
    
    for i in range(num_vectors):
        for j in range(num_vectors):
            list_of_tuples.append((i,j))
            
    return list_of_tuples

def pairwise_distance_from_indexes(indexes ):
    """takes a tuple containing a pair of  indexes, and computes the distance between the 2"""
    assert(len(indexes) == 2)
    X1 = data[indexes[0]]
    X2 = data[indexes[1]]
    
    return np.sqrt( np.sum( np.square( X1-X2 ) ) )


list_of_tuples_I = pairwise_list_I(data)

%timeit -n 1 -r 3  result = list(map(pairwise_distance_from_indexes,  list_of_tuples_I))

import multiprocessing as mp

with mp.Pool(2) as pool :
    
    %timeit -n 1 -r 3  result2 = pool.map(pairwise_distance_from_indexes, list_of_tuples_I)

Some speedup, but nothing tremendous.

Let's see if that holds up :

In [ ]:
for NP in [1,2,3,4,5,6]:
    print(NP)
    with mp.Pool(NP) as pool :
        %time result = pool.map(pairwise_distance_from_indexes, list_of_tuples_I)

In [ ]:
# Of course, we want to compare this with the original version of the function
%timeit pairwise_distance_numpy( data )

So there is some gain. 
Nothing tremendous, but sill a 1.5x speedup, and it beats the alternative of having all these core idle.

<br>

Going one step further, we know multiprocessing works better when the task are somewhat large. So, instead of say that a task is "compute a single distance", and having NxN tasks, we could have the task be "compute a full row of the distance matrix", and then we only have N tasks.

So for the task, we compute the distance between one vector and all the others. You will see this is a very good idea, even in a non-multiprocessing framework, because this plays into some of numpy's strength.

First, let's implement our "task":

In [ ]:
# Computing reference results for testing validity.
toy_data = np.random.uniform(size=(10,100))

res = pairwise_distance_numpy( toy_data )
# We want our task to compute something like this:
res[0]


In [ ]:
# We have seen that numpy makes operation between 2 vector easy.
# but actually, operation between a matrix and a vector works as well
# so, matrix - vector will perform the subtraction on each row independently.
# then, if we make the sum also on each row independently we can get the distances we want!

def compute_distance_row( i ):
    ## Here, I presume that there exists a DATA_GLOB
    ##  variable in global memory with my data in it
    
    squared_diff = ( DATA_GLOB - DATA_GLOB[i])**2 ## squared differences between the matrix and a single vector 
    sums = np.sum( squared_diff , axis = 1) ## axis=1 --> to get 1 sum per row
    return np.sqrt( sums ) ## compute square root of all these sums

## of course we want to test this:
DATA_GLOB = toy_data
res_new = compute_distance_row( 0 )
print(res_new)
print(res_new == res[0])

All good so far. How does it perform?

In [ ]:
DATA_GLOB = data  # redefine the data used by the function as the bigger dataset

## we map this onto the list of possible indices : from 0 to N
%timeit -n 1 -r 3  result = list(map(compute_distance_row, range(DATA_GLOB.shape[0])))

So you can see how this actually performs even better even with a single process.

Actually, let's make the data larger.

In [ ]:
big_data = np.random.uniform(size=(500,100))

In [ ]:
DATA_GLOB=big_data
%timeit -n 3 -r 3  result = list(map(compute_distance_row, range(DATA_GLOB.shape[0])))
for NP in [1,2,3,4,5,6]:
    print(NP)
    with mp.Pool(NP) as pool :
        %time result = pool.map(compute_distance_row, range(DATA_GLOB.shape[0]))

Same as before: with larger individual takes we seem to get better speedup in general (~x2 speedup for 4processes).

<br>
<br>

[Back to ToC](#toc)

## Annex 2 - parallelization of pairwise distance computation with numba <a id='annexb'></a>

In [ ]:
from numba import njit, prange

# njit -> no-python jit

@njit(parallel=True)
def pairwise_distance_numba_prange(X):

    num_vectors = X.shape[0]
    num_measurements = X.shape[1] 
    D = np.empty((num_vectors, num_vectors), dtype=np.float64)
    
    for i in prange(num_vectors): # note usage of prange
        for j in range(num_vectors):
            d = 0.
            for k in range(num_measurements):
                d += np.square(np.subtract(X[i][k], X[j][k]))
            D[i, j] = np.sqrt(d)
    return(D)

toy_data = np.random.uniform(size=(10,10))  # I make toy data to launch the function once and compile it
toy_result = pairwise_distance_numba_prange(toy_data) 

In [ ]:
print("numba parallel=True")
%timeit -n 1 -r 3 result = pairwise_distance_numba_prange(big_data)